In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import requests, json
import datetime
import time
import pytz

import alpaca_trade_api as api

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas
import numpy as np
from scipy import stats


In [3]:
API_KEY = 'AKCBVDALZTXXE0NJ7LMV'
SECRET_KEY = 'OgCPIyKsZ9g1iLrmU89uKcWUCPW59E8Vrc6bGzwz'


In [4]:
alpaca = api.REST(API_KEY, SECRET_KEY)

In [5]:
def generate_dates(start_date, end_date, start_hour=9, start_minute=30, end_hour=12, end_minute=0):
    dates_list = []
    for d in pandas.date_range(start=start_date, end=end_date):
        if d.weekday() > 4:
            continue
        start = pytz.timezone('US/Eastern').localize(d.replace(hour=start_hour, minute=start_minute)).astimezone(pytz.utc).isoformat()
        end = pytz.timezone('US/Eastern').localize(d.replace(hour=end_hour, minute=end_minute)).astimezone(pytz.utc).isoformat()
        dates_list.append((d.strftime("%Y-%m-%d"), start, end))
    return dates_list

In [6]:
def get_quotes(ticker, start, end, name=None):
    quotes = alpaca.get_quotes([ticker], start, end).df
    quotes = quotes.loc[~quotes.index.duplicated(keep='last')]
    
    if not quotes.empty:
        quotes = quotes[quotes['ask_price'] - quotes['bid_price'] > 0.0]
    
    return quotes


In [7]:
start_date = '2022-12-17'
end_date = '2023-02-02'

dates = generate_dates(start_date, end_date)


In [8]:
for i, date in enumerate(dates):
    print(f'{i} / {len(dates)}')
        
    date_string = date[0]
    
    from_iso = date[1]
    to_iso = date[2]
    
    q = get_quotes('SPY', from_iso, to_iso)
    q.to_pickle(f'quotes/SPY_{date_string}.pickle')


0 / 34


ConnectionError: HTTPSConnectionPool(host='data.alpaca.markets', port=443): Max retries exceeded with url: /v2/stocks/quotes?start=2022-12-19T14%3A30%3A00%2B00%3A00&end=2022-12-19T17%3A00%3A00%2B00%3A00&symbols=SPY (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000000000D689340>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))